In [65]:
import pandas as pd
import numpy as np

patients = pd.read_csv("data/PATIENTS.csv", low_memory=False)
diagnoses_icd = pd.read_csv("data/DIAGNOSES_ICD.csv", low_memory=False)
chartevents = pd.read_csv("data/CHARTEVENTS.csv", low_memory=False)
prescriptions = pd.read_csv("data/PRESCRIPTIONS.csv", low_memory=False)
labevents = pd.read_csv("data/LABEVENTS.csv", low_memory=False)
mapped_items = pd.read_csv("data/mapped_items.csv", low_memory=False)
mapped_icd9 = pd.read_csv("data/icd9codes_mapped.csv", low_memory=False)
vocabulary = pd.read_csv("data/vocabulary.csv", low_memory=False, keep_default_na=False)
domain = pd.read_csv("data/domain.csv", low_memory=False, keep_default_na=False)
mapped_drugs = pd.read_csv("data/alldrugs_mapped.csv", low_memory=False, keep_default_na=False)
drug_rxnorm = pd.read_csv("data/drug_rxnorm.csv", low_memory=False, keep_default_na=False)
mapped_additional_concepts = pd.read_csv("data/mapped_additional_concepts.csv", low_memory=False, keep_default_na=False, sep=';')

In [66]:
htn_diagnoses = diagnoses_icd[
    diagnoses_icd['icd9_code'].str.startswith(('401', '402', '403', '404', '405'))
]

print(htn_diagnoses.head())

    row_id  subject_id  hadm_id  seq_num icd9_code
3   112347       10006   142345        4     40391
49  112457       10017   199207       14      4019
61  112469       10019   177759       12      4019
65  112507       10026   103770        2      4010
78  112520       10027   199395        8      4019


In [67]:
d_items = pd.read_csv("data/D_ITEMS.csv") 


bp_items = d_items[
    d_items['label'].str.contains('arterial pressure|blood pressure', case=False, na=False)
]

print(bp_items[['itemid', 'label']])

       itemid                                     label
58         53                         Arterial Pressure
9528   220050          Arterial Blood Pressure systolic
9529   220051         Arterial Blood Pressure diastolic
9530   220052              Arterial Blood Pressure mean
9531   220056       Arterial Blood Pressure Alarm - Low
9532   220058      Arterial Blood Pressure Alarm - High
9546   220179      Non Invasive Blood Pressure systolic
9547   220180     Non Invasive Blood Pressure diastolic
9548   220181          Non Invasive Blood Pressure mean
9563   223752   Non-Invasive Blood Pressure Alarm - Low
9821   223751  Non-Invasive Blood Pressure Alarm - High
9862   224167       Manual Blood Pressure Systolic Left
10236  224643      Manual Blood Pressure Diastolic Left
11430  227242     Manual Blood Pressure Diastolic Right
11431  227243      Manual Blood Pressure Systolic Right
11595  227537           ART Blood Pressure Alarm - High
11596  227538            ART Blood Pressure Alar

In [68]:
bp_item_ids = bp_items['itemid'].unique()

bp_data = chartevents[chartevents['itemid'].isin(bp_item_ids)]
print(bp_data.head())

     row_id  subject_id  hadm_id  icustay_id  itemid            charttime  \
4   5279025       40124   126179    279554.0  220179  2130-02-04 04:32:00   
5   5279026       40124   126179    279554.0  220180  2130-02-04 04:32:00   
6   5279027       40124   126179    279554.0  220181  2130-02-04 04:32:00   
9   5279030       40124   126179    279554.0  223751  2130-02-04 04:34:00   
10  5279031       40124   126179    279554.0  223752  2130-02-04 04:34:00   

              storetime   cgid value  valuenum valueuom  warning  error  \
4   2130-02-04 04:43:00  21452   163     163.0     mmHg      0.0    0.0   
5   2130-02-04 04:43:00  21452    81      81.0     mmHg      0.0    0.0   
6   2130-02-04 04:43:00  21452   101     101.0     mmHg      0.0    0.0   
9   2130-02-04 04:34:00  19085   170     170.0     mmHg      0.0    0.0   
10  2130-02-04 04:34:00  19085    90      90.0     mmHg      0.0    0.0   

   resultstatus stopped  
4           NaN     NaN  
5           NaN     NaN  
6       

In [69]:
htn_drugs = prescriptions[
    prescriptions['drug'].str.contains('amlodipine|losartan|hydrochlorothiazide|metoprolol', case=False, na=False)
]

print(htn_drugs.head())

    row_id  subject_id  hadm_id  icustay_id            startdate  \
12   32612       42458   159647         NaN  2146-07-21 00:00:00   
19   59641       42275   128293    290478.0  2135-10-25 00:00:00   
49   86579       43735   112662    293363.0  2128-11-05 00:00:00   
56   86586       43735   112662         NaN  2128-11-07 00:00:00   
59   86589       43735   112662         NaN  2128-11-08 00:00:00   

                enddate drug_type                     drug  \
12  2146-07-22 00:00:00      MAIN  Metoprolol Succinate XL   
19  2135-10-27 00:00:00      MAIN               Amlodipine   
49  2128-11-09 00:00:00      MAIN      Metoprolol Tartrate   
56  2128-11-08 00:00:00      MAIN      Metoprolol Tartrate   
59  2128-11-09 00:00:00      MAIN      Metoprolol Tartrate   

              drug_name_poe        drug_name_generic formulary_drug_cd  \
12  Metoprolol Succinate XL  Metoprolol Succinate XL            TOPR25   
19               Amlodipine               Amlodipine             AMLO5

In [70]:
d_labitems = pd.read_csv("data/D_LABITEMS.csv")  

relevant_tests = d_labitems[
    d_labitems['label'].str.contains('sodium|potassium|creatinine|cholesterol', case=False, na=False)
]

print(relevant_tests[['itemid', 'label']])

test_item_ids = relevant_tests['itemid'].unique()
lab_data = labevents[labevents['itemid'].isin(test_item_ids)]
print(lab_data.head())

     itemid                            label
22    50822           Potassium, Whole Blood
24    50824              Sodium, Whole Blood
33    50833                        Potassium
34    50834               Sodium, Body Fluid
40    50840             Cholesterol, Ascites
41    50841              Creatinine, Ascites
47    50847               Potassium, Ascites
48    50848                  Sodium, Ascites
103   50903    Cholesterol Ratio (Total/HDL)
104   50904                 Cholesterol, HDL
105   50905     Cholesterol, LDL, Calculated
106   50906       Cholesterol, LDL, Measured
107   50907               Cholesterol, Total
112   50912                       Creatinine
171   50971                        Potassium
183   50983                           Sodium
220   51021          Creatinine, Joint Fluid
230   51031          Cholesterol, Body Fluid
231   51032           Creatinine, Body Fluid
240   51041            Potassium, Body Fluid
241   51042               Sodium, Body Fluid
250   5105

In [71]:
print(bp_data.isnull().sum()) 


bp_data = bp_data.dropna(subset=['value'])

row_id              0
subject_id          0
hadm_id             0
icustay_id          0
itemid              0
charttime           0
storetime           0
cgid                0
value               0
valuenum            7
valueuom            7
warning             0
error               0
resultstatus    25682
stopped         25682
dtype: int64


In [72]:
bp_data['charttime'] = pd.to_datetime(bp_data['charttime'])


bp_data['value'] = pd.to_numeric(bp_data['value'], errors='coerce')

In [73]:
bp_data

,row_id,subject_id,hadm_id,icustay_id,itemid,charttime,storetime,cgid,value,valuenum,valueuom,warning,error,resultstatus,stopped
4,5279025,40124,126179,279554.0,220179,2130-02-04 04:32:00,2130-02-04 04:43:00,21452,163.0,163.0,mmHg,0.0,0.0,NaN,NaN
5,5279026,40124,126179,279554.0,220180,2130-02-04 04:32:00,2130-02-04 04:43:00,21452,81.0,81.0,mmHg,0.0,0.0,NaN,NaN
6,5279027,40124,126179,279554.0,220181,2130-02-04 04:32:00,2130-02-04 04:43:00,21452,101.0,101.0,mmHg,0.0,0.0,NaN,NaN
9,5279030,40124,126179,279554.0,223751,2130-02-04 04:34:00,2130-02-04 04:34:00,19085,170.0,170.0,mmHg,0.0,0.0,NaN,NaN
10,5279031,40124,126179,279554.0,223752,2130-02-04 04:34:00,2130-02-04 04:34:00,19085,90.0,90.0,mmHg,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
578211,272565345,41976,176016,286072.0,227539,2199-02-01 16:04:00,2199-02-01 16:05:00,17446,NaN,NaN,NaN,0.0,0.0,NaN,NaN
578981,272566115,41976,176016,286072.0,227539,2199-02-02 08:07:00,2199-02-02 08:08:00,15843,NaN,NaN,NaN,0.0,0.0,NaN,NaN
584378,272571512,41976,176016,286072.0,227539,2199-02-06 21:52:00,2199-02-06 21:53:00,14772,NaN,NaN,NaN,0.0,0.0,NaN,NaN
675098,272949568,42367,139932,250305.0,227539,2147-10-12 08:00:00,2147-10-12 08:01:00,17346,NaN,NaN,NaN,0.0,0.0,NaN,NaN


In [74]:
patients_data = patients[['subject_id', 'gender', 'dob']]

In [75]:
bp_data = bp_data[['row_id','subject_id','hadm_id','itemid','charttime', 'value', 'valueuom']]

In [76]:
import pandas as pd

# Charger la table ADMISSIONS
admissions = pd.read_csv("data/ADMISSIONS.csv")

# Afficher les premières lignes pour un aperçu
print(admissions.head())

   row_id  subject_id  hadm_id            admittime            dischtime  \
0   12258       10006   142345  2164-10-23 21:09:00  2164-11-01 17:15:00   
1   12263       10011   105331  2126-08-14 22:32:00  2126-08-28 18:59:00   
2   12265       10013   165520  2125-10-04 23:36:00  2125-10-07 15:13:00   
3   12269       10017   199207  2149-05-26 17:19:00  2149-06-03 18:42:00   
4   12270       10019   177759  2163-05-14 20:43:00  2163-05-15 12:00:00   

             deathtime admission_type         admission_location  \
0                  NaN      EMERGENCY       EMERGENCY ROOM ADMIT   
1  2126-08-28 18:59:00      EMERGENCY  TRANSFER FROM HOSP/EXTRAM   
2  2125-10-07 15:13:00      EMERGENCY  TRANSFER FROM HOSP/EXTRAM   
3                  NaN      EMERGENCY       EMERGENCY ROOM ADMIT   
4  2163-05-15 12:00:00      EMERGENCY  TRANSFER FROM HOSP/EXTRAM   

  discharge_location insurance language  religion marital_status  \
0   HOME HEALTH CARE  Medicare      NaN  CATHOLIC      SEPARATED  

In [77]:
admissions_data = admissions[['row_id','subject_id', 'hadm_id', 'dischtime', 'admittime', 'admission_type', 'diagnosis']]

In [78]:
prescriptions_data = prescriptions[['row_id','subject_id', 'hadm_id', 'startdate', 'enddate', 'drug', 'drug_type','formulary_drug_cd','dose_val_rx','dose_unit_rx', 'route']]

In [79]:
htn_diagnoses = diagnoses_icd[['row_id','subject_id', 'hadm_id', 'seq_num', 'icd9_code']]


htn_diagnoses = htn_diagnoses.merge(
    admissions_data[['hadm_id', 'admittime', 'dischtime']],
    on='hadm_id',
    how='left'
)

htn_diagnoses

,row_id,subject_id,hadm_id,seq_num,icd9_code,admittime,dischtime
0,112344,10006,142345,1,99591,2164-10-23 21:09:00,2164-11-01 17:15:00
1,112345,10006,142345,2,99662,2164-10-23 21:09:00,2164-11-01 17:15:00
2,112346,10006,142345,3,5672,2164-10-23 21:09:00,2164-11-01 17:15:00
3,112347,10006,142345,4,40391,2164-10-23 21:09:00,2164-11-01 17:15:00
4,112348,10006,142345,5,42731,2164-10-23 21:09:00,2164-11-01 17:15:00
...,...,...,...,...,...,...,...
1756,397673,44228,103379,7,1975,2170-12-15 03:14:00,2170-12-24 18:00:00
1757,397674,44228,103379,8,45182,2170-12-15 03:14:00,2170-12-24 18:00:00
1758,397675,44228,103379,9,99592,2170-12-15 03:14:00,2170-12-24 18:00:00
1759,397676,44228,103379,10,2449,2170-12-15 03:14:00,2170-12-24 18:00:00


In [80]:
lab_data = labevents[['row_id','subject_id','hadm_id','itemid','charttime', 'value', 'valueuom']]

In [81]:
# Joindre htn_diagnoses avec admissions_data
htn_with_admissions = htn_diagnoses.merge(
    admissions_data,
    on=['subject_id', 'hadm_id'],  # Clés communes
    how='inner'
)

htn_with_admissions

,row_id_x,subject_id,hadm_id,seq_num,icd9_code,admittime_x,dischtime_x,row_id_y,dischtime_y,admittime_y,admission_type,diagnosis
0,112344,10006,142345,1,99591,2164-10-23 21:09:00,2164-11-01 17:15:00,12258,2164-11-01 17:15:00,2164-10-23 21:09:00,EMERGENCY,SEPSIS
1,112345,10006,142345,2,99662,2164-10-23 21:09:00,2164-11-01 17:15:00,12258,2164-11-01 17:15:00,2164-10-23 21:09:00,EMERGENCY,SEPSIS
2,112346,10006,142345,3,5672,2164-10-23 21:09:00,2164-11-01 17:15:00,12258,2164-11-01 17:15:00,2164-10-23 21:09:00,EMERGENCY,SEPSIS
3,112347,10006,142345,4,40391,2164-10-23 21:09:00,2164-11-01 17:15:00,12258,2164-11-01 17:15:00,2164-10-23 21:09:00,EMERGENCY,SEPSIS
4,112348,10006,142345,5,42731,2164-10-23 21:09:00,2164-11-01 17:15:00,12258,2164-11-01 17:15:00,2164-10-23 21:09:00,EMERGENCY,SEPSIS
...,...,...,...,...,...,...,...,...,...,...,...,...
1756,397673,44228,103379,7,1975,2170-12-15 03:14:00,2170-12-24 18:00:00,41092,2170-12-24 18:00:00,2170-12-15 03:14:00,EMERGENCY,CHOLANGITIS
1757,397674,44228,103379,8,45182,2170-12-15 03:14:00,2170-12-24 18:00:00,41092,2170-12-24 18:00:00,2170-12-15 03:14:00,EMERGENCY,CHOLANGITIS
1758,397675,44228,103379,9,99592,2170-12-15 03:14:00,2170-12-24 18:00:00,41092,2170-12-24 18:00:00,2170-12-15 03:14:00,EMERGENCY,CHOLANGITIS
1759,397676,44228,103379,10,2449,2170-12-15 03:14:00,2170-12-24 18:00:00,41092,2170-12-24 18:00:00,2170-12-15 03:14:00,EMERGENCY,CHOLANGITIS


In [82]:
# Joindre bp_data avec admissions_data
bp_with_admissions = bp_data.merge(
    admissions_data,
    on=['subject_id', 'hadm_id'],  # Clés communes
    how='inner'
)

# Filtrer les mesures prises pendant l'hospitalisation
bp_with_admissions = bp_with_admissions[
    (bp_with_admissions['charttime'] >= bp_with_admissions['admittime'])
]

bp_with_admissions

,row_id_x,subject_id,hadm_id,itemid,charttime,value,valueuom,row_id_y,dischtime,admittime,admission_type,diagnosis
0,5279025,40124,126179,220179,2130-02-04 04:32:00,163.0,mmHg,39821,2130-02-10 17:39:00,2130-02-04 02:26:00,EMERGENCY,SHORTNESS OF BREATH
1,5279026,40124,126179,220180,2130-02-04 04:32:00,81.0,mmHg,39821,2130-02-10 17:39:00,2130-02-04 02:26:00,EMERGENCY,SHORTNESS OF BREATH
2,5279027,40124,126179,220181,2130-02-04 04:32:00,101.0,mmHg,39821,2130-02-10 17:39:00,2130-02-04 02:26:00,EMERGENCY,SHORTNESS OF BREATH
3,5279030,40124,126179,223751,2130-02-04 04:34:00,170.0,mmHg,39821,2130-02-10 17:39:00,2130-02-04 02:26:00,EMERGENCY,SHORTNESS OF BREATH
4,5279031,40124,126179,223752,2130-02-04 04:34:00,90.0,mmHg,39821,2130-02-10 17:39:00,2130-02-04 02:26:00,EMERGENCY,SHORTNESS OF BREATH
...,...,...,...,...,...,...,...,...,...,...,...,...
25677,7469173,44228,103379,220180,2170-12-19 17:06:00,53.0,mmHg,41092,2170-12-24 18:00:00,2170-12-15 03:14:00,EMERGENCY,CHOLANGITIS
25678,7469174,44228,103379,220181,2170-12-19 17:06:00,67.0,mmHg,41092,2170-12-24 18:00:00,2170-12-15 03:14:00,EMERGENCY,CHOLANGITIS
25679,7469178,44228,103379,220179,2170-12-19 18:24:00,111.0,mmHg,41092,2170-12-24 18:00:00,2170-12-15 03:14:00,EMERGENCY,CHOLANGITIS
25680,7469179,44228,103379,220180,2170-12-19 18:24:00,54.0,mmHg,41092,2170-12-24 18:00:00,2170-12-15 03:14:00,EMERGENCY,CHOLANGITIS


In [83]:
# Joindre lab_data avec admissions_data
lab_with_admissions = lab_data.merge(
    admissions_data,
    on=['subject_id', 'hadm_id'],  # Clés communes
    how='inner'
)

# Filtrer les résultats obtenus pendant l'hospitalisation
lab_with_admissions = lab_with_admissions[
    (lab_with_admissions['charttime'] >= lab_with_admissions['admittime'])
]

lab_with_admissions


,row_id_x,subject_id,hadm_id,itemid,charttime,value,valueuom,row_id_y,dischtime,admittime,admission_type,diagnosis
65,6245263,10006,142345.0,50862,2164-10-24 03:00:00,2.7,g/dL,12258,2164-11-01 17:15:00,2164-10-23 21:09:00,EMERGENCY,SEPSIS
66,6245264,10006,142345.0,50868,2164-10-24 03:00:00,12,mEq/L,12258,2164-11-01 17:15:00,2164-10-23 21:09:00,EMERGENCY,SEPSIS
67,6245265,10006,142345.0,50882,2164-10-24 03:00:00,31,mEq/L,12258,2164-11-01 17:15:00,2164-10-23 21:09:00,EMERGENCY,SEPSIS
68,6245266,10006,142345.0,50893,2164-10-24 03:00:00,8.5,mg/dL,12258,2164-11-01 17:15:00,2164-10-23 21:09:00,EMERGENCY,SEPSIS
69,6245267,10006,142345.0,50902,2164-10-24 03:00:00,100,mEq/L,12258,2164-11-01 17:15:00,2164-10-23 21:09:00,EMERGENCY,SEPSIS
...,...,...,...,...,...,...,...,...,...,...,...,...
61807,20452679,44228,103379.0,51250,2170-12-24 04:09:00,88,fL,41092,2170-12-24 18:00:00,2170-12-15 03:14:00,EMERGENCY,CHOLANGITIS
61808,20452680,44228,103379.0,51265,2170-12-24 04:09:00,595,K/uL,41092,2170-12-24 18:00:00,2170-12-15 03:14:00,EMERGENCY,CHOLANGITIS
61809,20452681,44228,103379.0,51277,2170-12-24 04:09:00,14.5,%,41092,2170-12-24 18:00:00,2170-12-15 03:14:00,EMERGENCY,CHOLANGITIS
61810,20452682,44228,103379.0,51279,2170-12-24 04:09:00,2.76,m/uL,41092,2170-12-24 18:00:00,2170-12-15 03:14:00,EMERGENCY,CHOLANGITIS


In [84]:
# Joindre prescriptions_data avec admissions_data
prescriptions_with_admissions = prescriptions_data.merge(
    admissions_data,
    on=['subject_id', 'hadm_id'],  # Clés communes
    how='inner'
)

prescriptions_with_admissions

,row_id_x,subject_id,hadm_id,startdate,enddate,drug,drug_type,formulary_drug_cd,dose_val_rx,dose_unit_rx,route,row_id_y,dischtime,admittime,admission_type,diagnosis
0,32600,42458,159647,2146-07-21 00:00:00,2146-07-22 00:00:00,Pneumococcal Vac Polyvalent,MAIN,PNEU25I,0.5,mL,IM,40565,2146-07-22 14:45:00,2146-07-21 14:45:00,EMERGENCY,PNEUMONIA
1,32601,42458,159647,2146-07-21 00:00:00,2146-07-22 00:00:00,Bisacodyl,MAIN,BISA5,10,mg,PO,40565,2146-07-22 14:45:00,2146-07-21 14:45:00,EMERGENCY,PNEUMONIA
2,32602,42458,159647,2146-07-21 00:00:00,2146-07-22 00:00:00,Bisacodyl,MAIN,BISA10R,10,mg,PR,40565,2146-07-22 14:45:00,2146-07-21 14:45:00,EMERGENCY,PNEUMONIA
3,32603,42458,159647,2146-07-21 00:00:00,2146-07-22 00:00:00,Senna,MAIN,SENN187,1,TAB,PO,40565,2146-07-22 14:45:00,2146-07-21 14:45:00,EMERGENCY,PNEUMONIA
4,32604,42458,159647,2146-07-21 00:00:00,2146-07-21 00:00:00,Docusate Sodium (Liquid),MAIN,DOCU100L,100,mg,PO,40565,2146-07-22 14:45:00,2146-07-21 14:45:00,EMERGENCY,PNEUMONIA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10393,3609913,42430,100969,2142-11-29 00:00:00,2142-11-30 00:00:00,Sodium Chloride 0.9% Flush,MAIN,NACLFLUSH,3,mL,IV,40554,2142-11-30 10:55:00,2142-11-26 21:20:00,EMERGENCY,CEREBROVASCULAR ACCIDENT
10394,3609914,42430,100969,2142-11-30 00:00:00,2142-11-30 00:00:00,Acetaminophen,MAIN,ACET650R,650,mg,PR,40554,2142-11-30 10:55:00,2142-11-26 21:20:00,EMERGENCY,CEREBROVASCULAR ACCIDENT
10395,3609915,42430,100969,2142-11-26 00:00:00,2142-11-27 00:00:00,0.9% Sodium Chloride,BASE,NS1000,1000,mL,IV,40554,2142-11-30 10:55:00,2142-11-26 21:20:00,EMERGENCY,CEREBROVASCULAR ACCIDENT
10396,3609916,42430,100969,2142-11-26 00:00:00,2142-11-27 00:00:00,D5W,BASE,HEPBASE,250,mL,IV,40554,2142-11-30 10:55:00,2142-11-26 21:20:00,EMERGENCY,CEREBROVASCULAR ACCIDENT


In [85]:
mapped_items

,sourceCode,sourceName,sourceFrequency,sourceAutoAssignedConceptIds,matchScore,targetConceptId,targetConceptName,targetVocabularyId,targetDomainId,targetStandardConcept,targetChildCount,targetParentCount,targetConceptClassId,targetConceptCode,targetValidStartDate,targetValidEndDate,targetInvalidReason
0,220181,mean Non-Invasive Blood Pressure,-1,NaN,0.928882,4108289,Non-invasive mean arterial pressure,SNOMED,Measurement,S,0,2,Observable Entity,251074006,20020131,20991231,NaN
1,227538,Low Arterial Blood Pressure Alarm,-1,NaN,0.750751,4298391,Arterial blood pressure,SNOMED,Measurement,S,5,2,Observable Entity,386534000,20030131,20991231,NaN
2,227537,High Arterial Blood Pressure Alarm,-1,NaN,0.746245,4298391,Arterial blood pressure,SNOMED,Measurement,S,5,2,Observable Entity,386534000,20030131,20991231,NaN
3,220058,High Arterial Blood Pressure Alarm,-1,NaN,0.746245,4298391,Arterial blood pressure,SNOMED,Measurement,S,5,2,Observable Entity,386534000,20030131,20991231,NaN
4,220052,Arterial Blood Pressure mean,-1,NaN,0.882389,4298391,Arterial blood pressure,SNOMED,Measurement,S,5,2,Observable Entity,386534000,20030131,20991231,NaN
5,220050,Arterial Blood Pressure systolic,-1,NaN,0.808792,4298391,Arterial blood pressure,SNOMED,Measurement,S,5,2,Observable Entity,386534000,20030131,20991231,NaN
6,220051,Arterial Blood Pressure diastolic,-1,NaN,0.802012,4298391,Arterial blood pressure,SNOMED,Measurement,S,5,2,Observable Entity,386534000,20030131,20991231,NaN
7,227242,Right Manual Blood Pressure Diastolic,-1,NaN,0.689660,4154790,Diastolic blood pressure,SNOMED,Measurement,S,11,2,Observable Entity,271650006,20020131,20991231,NaN
8,220180,Non-Invasive Blood Pressure diastolic,-1,NaN,0.959331,4068414,Non-invasive diastolic arterial pressure,SNOMED,Measurement,S,0,1,Observable Entity,174255007,20020131,20991231,NaN
9,223751,High Non-Invasive Blood Pressure Alarm,-1,NaN,0.834386,36716965,Non-invasive blood pressure,SNOMED,Measurement,S,1,1,Observable Entity,723237002,20170131,20991231,NaN


In [86]:
concept = pd.DataFrame({
    'concept_id': mapped_items['targetConceptId'],
    'concept_name': mapped_items['targetConceptName'],
    'concept_code': mapped_items['targetConceptCode'],
    'domain_id': mapped_items['targetDomainId'],
    'vocabulary_id': mapped_items['targetVocabularyId'],
    'concept_class_id': mapped_items['targetConceptClassId'],
    'standard_concept': mapped_items['targetStandardConcept'],
    'valid_start_date': mapped_items['targetValidStartDate'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d', errors='coerce')),
    'valid_end_date': mapped_items['targetValidEndDate'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d', errors='coerce')),
    'invalid_reason': mapped_items['targetInvalidReason']
})

concept = concept.drop_duplicates()

concept

,concept_id,concept_name,concept_code,domain_id,vocabulary_id,concept_class_id,standard_concept,valid_start_date,valid_end_date,invalid_reason
0,4108289,Non-invasive mean arterial pressure,251074006,Measurement,SNOMED,Observable Entity,S,2002-01-31,2099-12-31,NaN
1,4298391,Arterial blood pressure,386534000,Measurement,SNOMED,Observable Entity,S,2003-01-31,2099-12-31,NaN
7,4154790,Diastolic blood pressure,271650006,Measurement,SNOMED,Observable Entity,S,2002-01-31,2099-12-31,NaN
8,4068414,Non-invasive diastolic arterial pressure,174255007,Measurement,SNOMED,Observable Entity,S,2002-01-31,2099-12-31,NaN
9,36716965,Non-invasive blood pressure,723237002,Measurement,SNOMED,Observable Entity,S,2017-01-31,2099-12-31,NaN
11,4354252,Non-invasive systolic arterial pressure,251070002,Measurement,SNOMED,Observable Entity,S,2002-01-31,2099-12-31,NaN
15,4152194,Systolic blood pressure,271649006,Measurement,SNOMED,Observable Entity,S,2002-01-31,2099-12-31,NaN


In [87]:
mapped_icd9

,sourceCode,sourceName,sourceFrequency,sourceAutoAssignedConceptIds,matchScore,targetConceptId,targetConceptName,targetVocabularyId,targetDomainId,targetStandardConcept,targetChildCount,targetParentCount,targetConceptClassId,targetConceptCode,targetValidStartDate,targetValidEndDate,targetInvalidReason
0,3669,cataract,-1,NaN,0.95,375545,Cataract,SNOMED,Condition,S,50,4,Disorder,193570009,20020131,20991231,NaN
1,2511,Other specified hypoglycemia,-1,NaN,0.74,24609,Hypoglycemia,SNOMED,Condition,S,4,2,Disorder,302866003,20020131,20991231,NaN
2,42830,Diastolic heart failure,-1,NaN,1.00,443587,Diastolic heart failure,SNOMED,Condition,S,5,3,Disorder,418304008,20060131,20991231,NaN
3,7580,Down's syndrome,-1,NaN,1.00,439125,Complete trisomy 21 syndrome,SNOMED,Condition,S,7,3,Disorder,41040004,20020131,20991231,NaN
4,V4986,Do not resuscitate status,-1,NaN,1.00,4119499,Not for resuscitation,SNOMED,Observation,S,2,1,Clinical Finding,304253006,20020131,20991231,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,1622,Malignant neoplasm of main bronchus,-1,NaN,1.00,4157333,Malignant neoplasm of main bronchus,SNOMED,Condition,S,2,2,Disorder,372065009,20020731,20991231,NaN
241,99682,Complications of transplanted liver,-1,NaN,0.93,200616,Complication of transplanted liver,SNOMED,Condition,S,5,3,Disorder,33167004,20020131,20991231,NaN
242,5712,Alcoholic cirrhosis of liver,-1,NaN,1.00,196463,Alcoholic cirrhosis,SNOMED,Condition,S,0,3,Disorder,420054005,20060131,20991231,NaN
243,5121,Iatrogenic pneumothorax,-1,NaN,1.00,40479583,Iatrogenic pneumothorax,SNOMED,Condition,S,0,2,Disorder,441536000,20090731,20991231,NaN


In [88]:
diagnostic_concept = pd.DataFrame({
    'concept_id': mapped_icd9['targetConceptId'],
    'concept_name': mapped_icd9['targetConceptName'],
    'concept_code': mapped_icd9['targetConceptCode'],
    'vocabulary_id': mapped_icd9['targetVocabularyId'],
    'concept_class_id': mapped_icd9['targetConceptClassId'],
    'standard_concept': mapped_icd9['targetStandardConcept'],
    'domain_id': mapped_icd9['targetDomainId'],
    'valid_start_date': mapped_icd9['targetValidStartDate'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d', errors='coerce')),
    'valid_end_date': mapped_icd9['targetValidEndDate'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d', errors='coerce')),
    'invalid_reason': mapped_icd9['targetInvalidReason']
})

diagnostic_concept = diagnostic_concept.drop_duplicates()

diagnostic_concept

,concept_id,concept_name,concept_code,vocabulary_id,concept_class_id,standard_concept,domain_id,valid_start_date,valid_end_date,invalid_reason
0,375545,Cataract,193570009,SNOMED,Disorder,S,Condition,2002-01-31,2099-12-31,NaN
1,24609,Hypoglycemia,302866003,SNOMED,Disorder,S,Condition,2002-01-31,2099-12-31,NaN
2,443587,Diastolic heart failure,418304008,SNOMED,Disorder,S,Condition,2006-01-31,2099-12-31,NaN
3,439125,Complete trisomy 21 syndrome,41040004,SNOMED,Disorder,S,Condition,2002-01-31,2099-12-31,NaN
4,4119499,Not for resuscitation,304253006,SNOMED,Clinical Finding,S,Observation,2002-01-31,2099-12-31,NaN
...,...,...,...,...,...,...,...,...,...,...
240,4157333,Malignant neoplasm of main bronchus,372065009,SNOMED,Disorder,S,Condition,2002-07-31,2099-12-31,NaN
241,200616,Complication of transplanted liver,33167004,SNOMED,Disorder,S,Condition,2002-01-31,2099-12-31,NaN
242,196463,Alcoholic cirrhosis,420054005,SNOMED,Disorder,S,Condition,2006-01-31,2099-12-31,NaN
243,40479583,Iatrogenic pneumothorax,441536000,SNOMED,Disorder,S,Condition,2009-07-31,2099-12-31,NaN


In [89]:
mapped_additional_concepts

,sourceCode,sourceName,sourceFrequency,sourceAutoAssignedConceptIds,matchScore,targetConceptId,targetConceptName,targetVocabularyId,targetDomainId,targetStandardConcept,targetChildCount,targetParentCount,targetConceptClassId,targetConceptCode,targetValidStartDate,targetValidEndDate,targetInvalidReason
0,,Male,-1,,1.0,8507,MALE,Gender,Gender,S,0,0,Gender,M,19700101,20991231,
1,,Organism,-1,,1.0,4259632,Organism,SNOMED,Observation,S,6,0,Organism,410607006,20040731,20991231,
2,,NEWBORN,-1,,1.0,4046034,Newborn,SNOMED,Relationship,S,0,1,Social Context,133933007,20020131,20991231,
3,,EMERGENCY,-1,,1.0,4093606,Emergency,SNOMED,Meas Value,S,1,1,Qualifier Value,25876001,20020131,20991231,
4,,ELECTIVE,-1,,1.0,4013731,Elective,SNOMED,Meas Value,S,1,1,Qualifier Value,103390000,20020131,20991231,
5,,Event,-1,,1.0,4156363,Event,SNOMED,Observation,S,42,0,Event,272379006,20020131,20991231,
6,,Quant Branded Drug,-1,,0.555040180683136,4224481,Larimus fasciatus,SNOMED,Observation,S,0,1,Organism,84868004,20020131,20991231,
7,,Female,-1,,1.0,8532,FEMALE,Gender,Gender,S,0,0,Gender,F,19700101,20991231,
8,,Clinical Finding,-1,,1.0,441840,Clinical finding,SNOMED,Observation,S,214,0,Clinical Finding,404684003,20040131,20991231,
9,,Morph Abnormality,-1,,0.6124445796012878,4219275,Sting,SNOMED,Observation,S,0,1,Morph Abnormality,72587008,20020131,20991231,


In [90]:
additionnal_concept = pd.DataFrame({
    'concept_id': mapped_additional_concepts['targetConceptId'],
    'concept_name': mapped_additional_concepts['targetConceptName'],
    'concept_code': mapped_additional_concepts['targetConceptCode'],
    'vocabulary_id': mapped_additional_concepts['targetVocabularyId'],
    'concept_class_id': mapped_additional_concepts['targetConceptClassId'],
    'standard_concept': mapped_additional_concepts['targetStandardConcept'],
    'domain_id': mapped_additional_concepts['targetDomainId'],
    'valid_start_date': mapped_additional_concepts['targetValidStartDate'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d', errors='coerce')),
    'valid_end_date': mapped_additional_concepts['targetValidEndDate'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d', errors='coerce')),
    'invalid_reason': mapped_additional_concepts['targetInvalidReason']
})

additionnal_concept = additionnal_concept.drop_duplicates()

additionnal_concept

,concept_id,concept_name,concept_code,vocabulary_id,concept_class_id,standard_concept,domain_id,valid_start_date,valid_end_date,invalid_reason
0,8507,MALE,M,Gender,Gender,S,Gender,1970-01-01,2099-12-31,
1,4259632,Organism,410607006,SNOMED,Organism,S,Observation,2004-07-31,2099-12-31,
2,4046034,Newborn,133933007,SNOMED,Social Context,S,Relationship,2002-01-31,2099-12-31,
3,4093606,Emergency,25876001,SNOMED,Qualifier Value,S,Meas Value,2002-01-31,2099-12-31,
4,4013731,Elective,103390000,SNOMED,Qualifier Value,S,Meas Value,2002-01-31,2099-12-31,
5,4156363,Event,272379006,SNOMED,Event,S,Observation,2002-01-31,2099-12-31,
6,4224481,Larimus fasciatus,84868004,SNOMED,Organism,S,Observation,2002-01-31,2099-12-31,
7,8532,FEMALE,F,Gender,Gender,S,Gender,1970-01-01,2099-12-31,
8,441840,Clinical finding,404684003,SNOMED,Clinical Finding,S,Observation,2004-01-31,2099-12-31,
9,4219275,Sting,72587008,SNOMED,Morph Abnormality,S,Observation,2002-01-31,2099-12-31,


In [91]:
concept_class_name = pd.Series(['Organism', 'Event', 'Larimus fasciatus','Clinical finding', 'Sting', 'Clinical observation regime', 'Finding with explicit context', 'Disease', 'Clinical drug trial', 'Observable entity'])
concept_class = pd.DataFrame({
    'concept_class_id': concept_class_name.map(additionnal_concept.set_index('concept_name')['concept_code']),
    'concept_class_name': concept_class_name,
    'concept_class_concept_id': concept_class_name.map(additionnal_concept.set_index('concept_name')['concept_id'])
})

concept_class

,concept_class_id,concept_class_name,concept_class_concept_id
0,410607006,Organism,4259632
1,272379006,Event,4156363
2,84868004,Larimus fasciatus,4224481
3,404684003,Clinical finding,441840
4,72587008,Sting,4219275
5,309466006,Clinical observation regime,4206580
6,413350009,Finding with explicit context,4187630
7,64572001,Disease,4274025
8,185922005,Clinical drug trial,4090378
9,363787002,Observable entity,4181663


In [92]:
mapped_drugs

,sourceCode,sourceName,sourceFrequency,sourceAutoAssignedConceptIds,matchScore,targetConceptId,targetConceptName,targetVocabularyId,targetDomainId,targetStandardConcept,targetChildCount,targetParentCount,targetConceptClassId,targetConceptCode,targetValidStartDate,targetValidEndDate,targetInvalidReason
0,5.107905e+10,Amlodipine,-1,,1.0,1332419,amlodipine 5 MG Oral Tablet,RxNorm,Drug,S,0,4,Clinical Drug,197361,19700101,20991231,
1,5.976215e+10,Amlodipine,-1,,1.0,1332419,amlodipine 5 MG Oral Tablet,RxNorm,Drug,S,0,4,Clinical Drug,197361,19700101,20991231,
2,6.915304e+07,Amlodipine,-1,,1.0,1332421,amlodipine 5 MG Oral Tablet [Norvasc],RxNorm,Drug,S,0,8,Branded Drug,212549,19700101,20991231,
3,6.258408e+10,Metoprolol,-1,,1.0,19019273,naproxen 500 MG Oral Tablet,RxNorm,Drug,S,2,4,Clinical Drug,198014,19700101,20991231,
4,6.033855e+08,Hydrochlorothiazide,-1,,1.0,19022486,hydrochlorothiazide 12.5 MG Oral Capsule,RxNorm,Drug,S,0,4,Clinical Drug,199903,19700101,20991231,
5,7.469780e+07,Hydrochlorothiazide,-1,,1.0,19036921,hydrochlorothiazide 25 MG Oral Tablet [Oretic],RxNorm,Drug,S,0,8,Branded Drug,207939,19700101,20991231,
6,1.861092e+08,Metoprolol XL,-1,,1.0,40166825,24 HR metoprolol succinate 100 MG Extended Rel...,RxNorm,Drug,S,0,8,Quant Branded Drug,866414,20091101,20991231,
7,5.817703e+10,Metoprolol Succinate XL,-1,,1.0,40166828,24 HR metoprolol succinate 25 MG Extended Rele...,RxNorm,Drug,S,0,4,Quant Clinical Drug,866427,20091101,20991231,
8,1.861088e+08,Metoprolol XL,-1,,1.0,40166829,24 HR metoprolol succinate 25 MG Extended Rele...,RxNorm,Drug,S,0,8,Quant Branded Drug,866429,20091101,20991231,
9,1.861090e+08,Metoprolol XL,-1,,1.0,40166831,24 HR metoprolol succinate 50 MG Extended Rele...,RxNorm,Drug,S,0,8,Quant Branded Drug,866438,20091101,20991231,


In [93]:
drug_concept = pd.DataFrame({
    'concept_id': mapped_drugs['targetConceptId'],
    'concept_name': mapped_drugs['targetConceptName'],
    'concept_code': mapped_drugs['targetConceptCode'],
    'vocabulary_id': mapped_drugs['targetVocabularyId'],
    'concept_class_id': mapped_drugs['targetConceptClassId'],
    'standard_concept': mapped_drugs['targetStandardConcept'],
    'domain_id': mapped_drugs['targetDomainId'],
    'valid_start_date': mapped_drugs['targetValidStartDate'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d', errors='coerce')),
    'valid_end_date': mapped_drugs['targetValidEndDate'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d', errors='coerce')),
    'invalid_reason': mapped_drugs['targetInvalidReason']
})

drug_concept = drug_concept.drop_duplicates()

drug_concept

,concept_id,concept_name,concept_code,vocabulary_id,concept_class_id,standard_concept,domain_id,valid_start_date,valid_end_date,invalid_reason
0,1332419,amlodipine 5 MG Oral Tablet,197361,RxNorm,Clinical Drug,S,Drug,1970-01-01,2099-12-31,
2,1332421,amlodipine 5 MG Oral Tablet [Norvasc],212549,RxNorm,Branded Drug,S,Drug,1970-01-01,2099-12-31,
3,19019273,naproxen 500 MG Oral Tablet,198014,RxNorm,Clinical Drug,S,Drug,1970-01-01,2099-12-31,
4,19022486,hydrochlorothiazide 12.5 MG Oral Capsule,199903,RxNorm,Clinical Drug,S,Drug,1970-01-01,2099-12-31,
5,19036921,hydrochlorothiazide 25 MG Oral Tablet [Oretic],207939,RxNorm,Branded Drug,S,Drug,1970-01-01,2099-12-31,
6,40166825,24 HR metoprolol succinate 100 MG Extended Rel...,866414,RxNorm,Quant Branded Drug,S,Drug,2009-11-01,2099-12-31,
7,40166828,24 HR metoprolol succinate 25 MG Extended Rele...,866427,RxNorm,Quant Clinical Drug,S,Drug,2009-11-01,2099-12-31,
8,40166829,24 HR metoprolol succinate 25 MG Extended Rele...,866429,RxNorm,Quant Branded Drug,S,Drug,2009-11-01,2099-12-31,
9,40166831,24 HR metoprolol succinate 50 MG Extended Rele...,866438,RxNorm,Quant Branded Drug,S,Drug,2009-11-01,2099-12-31,
11,40167196,5 ML metoprolol tartrate 1 MG/ML Injection,866508,RxNorm,Quant Clinical Drug,S,Drug,2009-11-01,2099-12-31,


In [94]:
gender_mapping = {
    'M': 8507,  # Masculin
    'F': 8532   # Féminin
}

# Construire la table standardisée `person`
person = pd.DataFrame({
    'person_id': patients_data['subject_id'],  # ID patient
    'gender_concept_id': patients_data['gender'].map(gender_mapping),  # Genre
    'year_of_birth': pd.to_datetime(patients_data['dob']).dt.year,  # Année de naissance
    'month_of_birth': pd.to_datetime(patients_data['dob']).dt.month,  # Mois de naissance
    'day_of_birth': pd.to_datetime(patients_data['dob']).dt.day,  # Date de décès
    'race_concept_id': -1,
    'ethnicity_concept_id': -1,
})

# Visualiser un aperçu
print(person.head())

   person_id  gender_concept_id  year_of_birth  month_of_birth  day_of_birth  \
0      10006               8532           2094               3             5   
1      10011               8532           2090               6             5   
2      10013               8532           2038               9             3   
3      10017               8532           2075               9            21   
4      10019               8507           2114               6            20   

   race_concept_id  ethnicity_concept_id  
0               -1                    -1  
1               -1                    -1  
2               -1                    -1  
3               -1                    -1  
4               -1                    -1  


In [95]:
print(person.isnull().sum())

# Vérifier les valeurs uniques pour le genre
print(person['gender_concept_id'].unique())

person_id               0
gender_concept_id       0
year_of_birth           0
month_of_birth          0
day_of_birth            0
race_concept_id         0
ethnicity_concept_id    0
dtype: int64
[8532 8507]


In [96]:
admission_type_mapping = {
    'EMERGENCY': 4093606,  # Urgence
    'ELECTIVE': 4013731,   # Élective
    'NEWBORN': 4046034 # Nouveau-né
}

# Construire la table `visit_occurrence`
visit_occurrence = pd.DataFrame({
    'visit_occurrence_id': admissions_data['hadm_id'],
    'person_id': admissions_data['subject_id'],
    'visit_concept_id': admissions_data['admission_type'].map(admission_type_mapping).fillna(-1).astype(int),
    'visit_type_concept_id': admissions_data['admission_type'].map(admission_type_mapping).fillna(-1).astype(int),
    'visit_start_datetime': pd.to_datetime(admissions_data['admittime']),
    'visit_start_date': pd.to_datetime(admissions_data['admittime']).dt.date,
    'visit_end_datetime': pd.to_datetime(admissions_data['dischtime']),
    'visit_end_date': pd.to_datetime(admissions_data['dischtime']).dt.date,
})

visit_occurrence.head()

,visit_occurrence_id,person_id,visit_concept_id,visit_type_concept_id,visit_start_datetime,visit_start_date,visit_end_datetime,visit_end_date
0,142345,10006,4093606,4093606,2164-10-23 21:09:00,2164-10-23,2164-11-01 17:15:00,2164-11-01
1,105331,10011,4093606,4093606,2126-08-14 22:32:00,2126-08-14,2126-08-28 18:59:00,2126-08-28
2,165520,10013,4093606,4093606,2125-10-04 23:36:00,2125-10-04,2125-10-07 15:13:00,2125-10-07
3,199207,10017,4093606,4093606,2149-05-26 17:19:00,2149-05-26,2149-06-03 18:42:00,2149-06-03
4,177759,10019,4093606,4093606,2163-05-14 20:43:00,2163-05-14,2163-05-15 12:00:00,2163-05-15


In [97]:
diagnostic_concept

,concept_id,concept_name,concept_code,vocabulary_id,concept_class_id,standard_concept,domain_id,valid_start_date,valid_end_date,invalid_reason
0,375545,Cataract,193570009,SNOMED,Disorder,S,Condition,2002-01-31,2099-12-31,NaN
1,24609,Hypoglycemia,302866003,SNOMED,Disorder,S,Condition,2002-01-31,2099-12-31,NaN
2,443587,Diastolic heart failure,418304008,SNOMED,Disorder,S,Condition,2006-01-31,2099-12-31,NaN
3,439125,Complete trisomy 21 syndrome,41040004,SNOMED,Disorder,S,Condition,2002-01-31,2099-12-31,NaN
4,4119499,Not for resuscitation,304253006,SNOMED,Clinical Finding,S,Observation,2002-01-31,2099-12-31,NaN
...,...,...,...,...,...,...,...,...,...,...
240,4157333,Malignant neoplasm of main bronchus,372065009,SNOMED,Disorder,S,Condition,2002-07-31,2099-12-31,NaN
241,200616,Complication of transplanted liver,33167004,SNOMED,Disorder,S,Condition,2002-01-31,2099-12-31,NaN
242,196463,Alcoholic cirrhosis,420054005,SNOMED,Disorder,S,Condition,2006-01-31,2099-12-31,NaN
243,40479583,Iatrogenic pneumothorax,441536000,SNOMED,Disorder,S,Condition,2009-07-31,2099-12-31,NaN


In [98]:
condition_occurrence = pd.DataFrame({
    'condition_occurrence_id': htn_diagnoses['row_id'],
    'person_id': htn_diagnoses['subject_id'],
    'condition_concept_id': htn_diagnoses['icd9_code'].map(mapped_icd9.set_index('sourceCode')['targetConceptId']).fillna(-1).astype(int),
    'condition_type_concept_id' : htn_diagnoses['icd9_code'].map(mapped_icd9.set_index('sourceCode')['targetConceptId']).map(diagnostic_concept.set_index('concept_id')['concept_class_id']).map(mapped_additional_concepts.set_index('sourceName')['targetConceptId']).fillna(-1).astype(int),
    'condition_start_datetime': pd.to_datetime(htn_diagnoses['admittime']),
    'condition_start_date': pd.to_datetime(htn_diagnoses['admittime']).dt.date,
    'condition_end_datetime': pd.to_datetime(htn_diagnoses['dischtime']),
    'condition_end_date': pd.to_datetime(htn_diagnoses['dischtime']).dt.date,
    'visit_occurrence_id': htn_diagnoses['hadm_id']
})

condition_occurrence

,condition_occurrence_id,person_id,condition_concept_id,condition_type_concept_id,condition_start_datetime,condition_start_date,condition_end_datetime,condition_end_date,visit_occurrence_id
0,112344,10006,132797,4274025,2164-10-23 21:09:00,2164-10-23,2164-11-01 17:15:00,2164-11-01,142345
1,112345,10006,-1,-1,2164-10-23 21:09:00,2164-10-23,2164-11-01 17:15:00,2164-11-01,142345
2,112346,10006,-1,-1,2164-10-23 21:09:00,2164-10-23,2164-11-01 17:15:00,2164-11-01,142345
3,112347,10006,-1,-1,2164-10-23 21:09:00,2164-10-23,2164-11-01 17:15:00,2164-11-01,142345
4,112348,10006,313217,4274025,2164-10-23 21:09:00,2164-10-23,2164-11-01 17:15:00,2164-11-01,142345
...,...,...,...,...,...,...,...,...,...
1756,397673,44228,-1,-1,2170-12-15 03:14:00,2170-12-15,2170-12-24 18:00:00,2170-12-24,103379
1757,397674,44228,433218,4274025,2170-12-15 03:14:00,2170-12-15,2170-12-24 18:00:00,2170-12-24,103379
1758,397675,44228,37394658,4274025,2170-12-15 03:14:00,2170-12-15,2170-12-24 18:00:00,2170-12-24,103379
1759,397676,44228,138384,4274025,2170-12-15 03:14:00,2170-12-15,2170-12-24 18:00:00,2170-12-24,103379


In [99]:
mapped_drugs = mapped_drugs.drop_duplicates(['sourceName'])

mapped_drugs

,sourceCode,sourceName,sourceFrequency,sourceAutoAssignedConceptIds,matchScore,targetConceptId,targetConceptName,targetVocabularyId,targetDomainId,targetStandardConcept,targetChildCount,targetParentCount,targetConceptClassId,targetConceptCode,targetValidStartDate,targetValidEndDate,targetInvalidReason
0,5.107905e+10,Amlodipine,-1,,1.0,1332419,amlodipine 5 MG Oral Tablet,RxNorm,Drug,S,0,4,Clinical Drug,197361,19700101,20991231,
3,6.258408e+10,Metoprolol,-1,,1.0,19019273,naproxen 500 MG Oral Tablet,RxNorm,Drug,S,2,4,Clinical Drug,198014,19700101,20991231,
4,6.033855e+08,Hydrochlorothiazide,-1,,1.0,19022486,hydrochlorothiazide 12.5 MG Oral Capsule,RxNorm,Drug,S,0,4,Clinical Drug,199903,19700101,20991231,
6,1.861092e+08,Metoprolol XL,-1,,1.0,40166825,24 HR metoprolol succinate 100 MG Extended Rel...,RxNorm,Drug,S,0,8,Quant Branded Drug,866414,20091101,20991231,
7,5.817703e+10,Metoprolol Succinate XL,-1,,1.0,40166828,24 HR metoprolol succinate 25 MG Extended Rele...,RxNorm,Drug,S,0,4,Quant Clinical Drug,866427,20091101,20991231,
11,1.439873e+08,Metoprolol Tartrate,-1,,1.0,40167196,5 ML metoprolol tartrate 1 MG/ML Injection,RxNorm,Drug,S,0,3,Quant Clinical Drug,866508,20091101,20991231,
20,6.095128e+06,Losartan Potassium,-1,,1.0,40185281,losartan potassium 25 MG Oral Tablet [Cozaar],RxNorm,Drug,S,0,8,Branded Drug,979487,20100606,20991231,


In [100]:
drug_exposure = pd.DataFrame({
    'drug_exposure_id': prescriptions_data['row_id'],
    'person_id': prescriptions_data['subject_id'],
    'drug_concept_id': prescriptions_data['drug'].map(mapped_drugs.set_index('sourceName')['targetConceptId']).fillna(-1).astype(int),
    'drug_type_concept_id': prescriptions_data['drug'].map(mapped_drugs.set_index('sourceName')['targetConceptId']).map(drug_concept.set_index('concept_id')['concept_class_id']).map(mapped_additional_concepts.set_index('sourceName')['targetConceptId']).fillna(-1).astype(int),
    'drug_exposure_start_datetime': pd.to_datetime(prescriptions_data['startdate']),
    'drug_exposure_start_date': pd.to_datetime(prescriptions_data['startdate']).dt.date,
    'drug_exposure_end_datetime': pd.to_datetime(prescriptions_data['enddate']).fillna(pd.NaT),
    'drug_exposure_end_date': pd.to_datetime(prescriptions_data['enddate']).dt.date.fillna(pd.to_datetime('1970-01-01')),
    'quantity': pd.to_numeric(prescriptions_data['dose_val_rx'], errors='coerce'),
    'dose_unit_source_value': prescriptions_data['dose_unit_rx'],
    'visit_occurrence_id': prescriptions_data['hadm_id']
})

drug_exposure[drug_exposure['drug_concept_id'] != -1]

,drug_exposure_id,person_id,drug_concept_id,drug_type_concept_id,drug_exposure_start_datetime,drug_exposure_start_date,drug_exposure_end_datetime,drug_exposure_end_date,quantity,dose_unit_source_value,visit_occurrence_id
12,32612,42458,40166828,4090378,2146-07-21,2146-07-21,2146-07-22,2146-07-22,12.5,mg,159647
19,59641,42275,1332419,4090378,2135-10-25,2135-10-25,2135-10-27,2135-10-27,5.0,mg,128293
49,86579,43735,40167196,4090378,2128-11-05,2128-11-05,2128-11-09,2128-11-09,12.5,mg,112662
56,86586,43735,40167196,4090378,2128-11-07,2128-11-07,2128-11-08,2128-11-08,5.0,mg,112662
59,86589,43735,40167196,4090378,2128-11-08,2128-11-08,2128-11-09,2128-11-09,5.0,mg,112662
...,...,...,...,...,...,...,...,...,...,...,...
10110,3540458,41983,40167196,4090378,2151-09-13,2151-09-13,2151-09-13,2151-09-13,5.0,mg,107689
10111,3540459,41983,40167196,4090378,2151-09-14,2151-09-14,2151-09-14,2151-09-14,5.0,mg,107689
10139,3559507,40612,40167196,4090378,2159-11-17,2159-11-17,2159-11-18,2159-11-18,NaN,mg,104697
10154,3559522,40612,40167196,4090378,2159-11-18,2159-11-18,2159-11-19,2159-11-19,2.5,mg,104697


In [101]:
measurement = pd.DataFrame({
    'measurement_id': bp_data['row_id'],
    'person_id': bp_data['subject_id'],
    'measurement_concept_id': bp_data['itemid'].map(mapped_items.set_index('sourceCode')['targetConceptId']).fillna(-1),
    "measurement_type_concept_id": bp_data['itemid'].map(mapped_items.set_index('sourceCode')['targetConceptClassId']).map(mapped_additional_concepts.set_index('sourceName')['targetConceptId']).fillna(-1).astype(int),
    'measurement_datetime': pd.to_datetime(bp_data['charttime']),
    'measurement_date': pd.to_datetime(bp_data['charttime']).dt.date,
    'value_as_number': pd.to_numeric(bp_data['value'], errors='coerce'),
    'unit_source_value': bp_data['valueuom']
})

measurement

,measurement_id,person_id,measurement_concept_id,measurement_type_concept_id,measurement_datetime,measurement_date,value_as_number,unit_source_value
4,5279025,40124,4354252,4181663,2130-02-04 04:32:00,2130-02-04,163.0,mmHg
5,5279026,40124,4068414,4181663,2130-02-04 04:32:00,2130-02-04,81.0,mmHg
6,5279027,40124,4108289,4181663,2130-02-04 04:32:00,2130-02-04,101.0,mmHg
9,5279030,40124,36716965,4181663,2130-02-04 04:34:00,2130-02-04,170.0,mmHg
10,5279031,40124,36716965,4181663,2130-02-04 04:34:00,2130-02-04,90.0,mmHg
...,...,...,...,...,...,...,...,...
578211,272565345,41976,4298391,4181663,2199-02-01 16:04:00,2199-02-01,NaN,NaN
578981,272566115,41976,4298391,4181663,2199-02-02 08:07:00,2199-02-02,NaN,NaN
584378,272571512,41976,4298391,4181663,2199-02-06 21:52:00,2199-02-06,NaN,NaN
675098,272949568,42367,4298391,4181663,2147-10-12 08:00:00,2147-10-12,NaN,NaN


In [102]:
print(visit_occurrence.head())
print(condition_occurrence.head())
print(drug_exposure.head())
print(measurement.head())

   visit_occurrence_id  person_id  visit_concept_id  visit_type_concept_id  \
0               142345      10006           4093606                4093606   
1               105331      10011           4093606                4093606   
2               165520      10013           4093606                4093606   
3               199207      10017           4093606                4093606   
4               177759      10019           4093606                4093606   

  visit_start_datetime visit_start_date  visit_end_datetime visit_end_date  
0  2164-10-23 21:09:00       2164-10-23 2164-11-01 17:15:00     2164-11-01  
1  2126-08-14 22:32:00       2126-08-14 2126-08-28 18:59:00     2126-08-28  
2  2125-10-04 23:36:00       2125-10-04 2125-10-07 15:13:00     2125-10-07  
3  2149-05-26 17:19:00       2149-05-26 2149-06-03 18:42:00     2149-06-03  
4  2163-05-14 20:43:00       2163-05-14 2163-05-15 12:00:00     2163-05-15  
   condition_occurrence_id  person_id  condition_concept_id  \
0     

In [103]:
print(visit_occurrence.dtypes)
print(condition_occurrence.dtypes)
print(drug_exposure.dtypes)
print(measurement.dtypes)

visit_occurrence_id               int64
person_id                         int64
visit_concept_id                  int64
visit_type_concept_id             int64
visit_start_datetime     datetime64[ns]
visit_start_date                 object
visit_end_datetime       datetime64[ns]
visit_end_date                   object
dtype: object
condition_occurrence_id               int64
person_id                             int64
condition_concept_id                  int64
condition_type_concept_id             int64
condition_start_datetime     datetime64[ns]
condition_start_date                 object
condition_end_datetime       datetime64[ns]
condition_end_date                   object
visit_occurrence_id                   int64
dtype: object
drug_exposure_id                         int64
person_id                                int64
drug_concept_id                          int64
drug_type_concept_id                     int64
drug_exposure_start_datetime    datetime64[ns]
drug_exposure_start_d

In [104]:
from sqlalchemy import create_engine
from sqlalchemy import text


# Define your database credentials
username = 'postgres'
password = 'mypass'
host = 'localhost'  # e.g., 'localhost' or an IP address
port = '5432'  # e.g., '5432' (default for PostgreSQL)
database = 'postgres'

# Create a connection string
connection_string = f'postgresql://{username}:{password}@{host}:{port}/{database}'

# Create the SQLAlchemy engine
engine = create_engine(connection_string)

# Test the connection
try:
    with engine.connect() as connection:
        print("Connected to the database successfully!")
except Exception as e:
    print("Failed to connect to the database.")
    print(e)

Connected to the database successfully!


In [105]:
truncate_command = """TRUNCATE demo_cdm.attribute_definition, 
        demo_cdm.care_site, 
        demo_cdm.cdm_source, 
        demo_cdm.cohort_definition, 
        demo_cdm.concept, 
        demo_cdm.concept_ancestor, 
        demo_cdm.concept_class, 
        demo_cdm.concept_recommended, 
        demo_cdm.concept_relationship, 
        demo_cdm.concept_synonym, 
        demo_cdm.condition_era, 
        demo_cdm.condition_occurrence, 
        demo_cdm.cost, 
        demo_cdm.death, 
        demo_cdm.device_exposure, 
        demo_cdm.domain, 
        demo_cdm.dose_era, 
        demo_cdm.drug_era, 
        demo_cdm.drug_exposure, 
        demo_cdm.drug_strength, 
        demo_cdm.fact_relationship, 
        demo_cdm.location, 
        demo_cdm.measurement, 
        demo_cdm.metadata, 
        demo_cdm.note, 
        demo_cdm.note_nlp, 
        demo_cdm.observation, 
        demo_cdm.observation_period, 
        demo_cdm.payer_plan_period, 
        demo_cdm.person, 
        demo_cdm.procedure_occurrence, 
        demo_cdm.provider, 
        demo_cdm.relationship, 
        demo_cdm.source_to_concept_map, 
        demo_cdm.specimen, 
        demo_cdm.visit_detail, 
        demo_cdm.visit_occurrence, 
        demo_cdm.vocabulary;"""

try:
    with engine.connect() as connection:
        with connection.begin():
            connection.execute(text(truncate_command))
            print("Truncated all tables.")
except Exception as e:
    print("Failed to truncate tables.")
    print(e)

Truncated all tables.


In [106]:
all_concepts = pd.concat([concept, diagnostic_concept, additionnal_concept, drug_concept])

In [107]:
visit_occurrence.to_sql('visit_occurrence', engine, schema='demo_cdm', if_exists='append', index=False)
condition_occurrence.to_sql('condition_occurrence', engine, schema='demo_cdm', if_exists='append', index=False)
drug_exposure.to_sql('drug_exposure', engine, schema='demo_cdm', if_exists='append', index=False)
measurement.to_sql('measurement', engine, schema='demo_cdm', if_exists='append', index=False)
person.to_sql('person', engine, schema='demo_cdm', if_exists='append', index=False)
vocabulary.to_sql('vocabulary', engine, schema='demo_cdm', if_exists='append', index=False)
domain.to_sql('domain', engine, schema='demo_cdm', if_exists='append', index=False)
concept_class.to_sql('concept_class', engine, schema='demo_cdm', if_exists='append', index=False)
all_concepts.to_sql('concept', engine, schema='demo_cdm', if_exists='append', index=False)

276

In [108]:
measurement['measurement_concept_id'].unique()

array([ 4354252,  4068414,  4108289, 36716965,  4298391,  4152194,
        4154790])

In [109]:
concept

,concept_id,concept_name,concept_code,domain_id,vocabulary_id,concept_class_id,standard_concept,valid_start_date,valid_end_date,invalid_reason
0,4108289,Non-invasive mean arterial pressure,251074006,Measurement,SNOMED,Observable Entity,S,2002-01-31,2099-12-31,NaN
1,4298391,Arterial blood pressure,386534000,Measurement,SNOMED,Observable Entity,S,2003-01-31,2099-12-31,NaN
7,4154790,Diastolic blood pressure,271650006,Measurement,SNOMED,Observable Entity,S,2002-01-31,2099-12-31,NaN
8,4068414,Non-invasive diastolic arterial pressure,174255007,Measurement,SNOMED,Observable Entity,S,2002-01-31,2099-12-31,NaN
9,36716965,Non-invasive blood pressure,723237002,Measurement,SNOMED,Observable Entity,S,2017-01-31,2099-12-31,NaN
11,4354252,Non-invasive systolic arterial pressure,251070002,Measurement,SNOMED,Observable Entity,S,2002-01-31,2099-12-31,NaN
15,4152194,Systolic blood pressure,271649006,Measurement,SNOMED,Observable Entity,S,2002-01-31,2099-12-31,NaN


In [110]:
print(concept['concept_class_id'].unique())
print(drug_concept['concept_class_id'].unique())
print(diagnostic_concept['concept_class_id'].unique())

['Observable Entity']
['Clinical Drug' 'Branded Drug' 'Quant Branded Drug' 'Quant Clinical Drug']
['Disorder' 'Clinical Finding' 'Context-dependent' 'Morph Abnormality'
 'Observable Entity' 'Event' 'Organism' 'Clinical Observation']


In [111]:
with engine.connect() as connect:
    result = connect.execute(text("SELECT * FROM demo_cdm.person"))
    for row in result:
        print(row)

(10006, 8532, 2094, 3, 5, None, -1, -1, None, None, None, None, None, None, None, None, None, None)
(10011, 8532, 2090, 6, 5, None, -1, -1, None, None, None, None, None, None, None, None, None, None)
(10013, 8532, 2038, 9, 3, None, -1, -1, None, None, None, None, None, None, None, None, None, None)
(10017, 8532, 2075, 9, 21, None, -1, -1, None, None, None, None, None, None, None, None, None, None)
(10019, 8507, 2114, 6, 20, None, -1, -1, None, None, None, None, None, None, None, None, None, None)
(10026, 8532, 1895, 5, 17, None, -1, -1, None, None, None, None, None, None, None, None, None, None)
(10027, 8532, 2108, 1, 15, None, -1, -1, None, None, None, None, None, None, None, None, None, None)
(10029, 8507, 2061, 4, 10, None, -1, -1, None, None, None, None, None, None, None, None, None, None)
(10032, 8507, 2050, 3, 29, None, -1, -1, None, None, None, None, None, None, None, None, None, None)
(10033, 8532, 2051, 4, 21, None, -1, -1, None, None, None, None, None, None, None, None, None,

In [112]:
request = text("""
SELECT
    -- Number of persons
    (SELECT COUNT(DISTINCT person_id) FROM demo_cdm.person) AS number_of_persons,
    
    -- Number of admissions
    (SELECT COUNT(*) FROM demo_cdm.visit_occurrence) AS number_of_admissions,
    
    -- Number of diagnostics (conditions)
    (SELECT COUNT(DISTINCT concept_id)
     FROM demo_cdm.concept
     WHERE domain_id = 'Condition') AS number_of_diagnostics_standardized,
    
    -- Number of prescriptions
    (SELECT COUNT(DISTINCT concept_id)
     FROM demo_cdm.concept 
     WHERE domain_id = 'Drug') AS number_of_prescriptions_standardized

""")

with engine.connect() as connect:
    result = connect.execute(request)
    column_names = result.keys()
    print(" | ".join(column_names))
    for row in result:
        print(row)

number_of_persons | number_of_admissions | number_of_diagnostics_standardized | number_of_prescriptions_standardized
(100, 129, 225, 13)


In [113]:
request = text("""
SELECT 
    p.gender_concept_id,
    c.concept_name AS gender_name,
    COUNT(*) * 100.0 / (SELECT COUNT(*) FROM demo_cdm.person) AS percentage
FROM 
    demo_cdm.person p
JOIN 
    demo_cdm.concept c
ON 
    p.gender_concept_id = c.concept_id
GROUP BY 
    p.gender_concept_id, c.concept_name;
""")

with engine.connect() as connection:
    result = connection.execute(request)
    column_names = result.keys()
    print(" | ".join(column_names))
    for row in result:
        print(row)  

gender_concept_id | gender_name | percentage
(8507, 'MALE', Decimal('45.0000000000000000'))
(8532, 'FEMALE', Decimal('55.0000000000000000'))


In [114]:
request = text("""
SELECT 
    d.drug_concept_id,
    c.concept_name AS drug_name,
    c.concept_class_id AS drug_class,
    COUNT(*) AS usage_count
FROM 
    demo_cdm.drug_exposure d
JOIN 
    demo_cdm.concept c
ON 
    d.drug_concept_id = c.concept_id
WHERE 
    d.drug_concept_id != -1
GROUP BY 
    d.drug_concept_id, c.concept_name, c.concept_class_id
ORDER BY 
    usage_count DESC;
""")

with engine.connect() as connection:
    result = connection.execute(request)
    column_names = result.keys()
    print(" | ".join(column_names))
    for row in result:
        print(row)

drug_concept_id | drug_name | drug_class | usage_count
(40167196, '5 ML metoprolol tartrate 1 MG/ML Injection', 'Quant Clinical Drug', 138)
(19019273, 'naproxen 500 MG Oral Tablet', 'Clinical Drug', 101)
(1332419, 'amlodipine 5 MG Oral Tablet', 'Clinical Drug', 10)
(19022486, 'hydrochlorothiazide 12.5 MG Oral Capsule', 'Clinical Drug', 8)
(40166825, '24 HR metoprolol succinate 100 MG Extended Release Oral Tablet [Toprol]', 'Quant Branded Drug', 4)
(40166828, '24 HR metoprolol succinate 25 MG Extended Release Oral Tablet', 'Quant Clinical Drug', 4)
(40185281, 'losartan potassium 25 MG Oral Tablet [Cozaar]', 'Branded Drug', 3)


In [115]:
request = text("""
SELECT 
    m.measurement_concept_id,
    c.concept_name AS measurement_name,
    COUNT(*) AS measurement_count
FROM 
    demo_cdm.measurement m
JOIN 
    demo_cdm.concept c
ON 
    m.measurement_concept_id = c.concept_id
WHERE 
    m.measurement_concept_id != -1
GROUP BY 
    m.measurement_concept_id, c.concept_name
ORDER BY 
    measurement_count DESC;
""")

with engine.connect() as connection:
    result = connection.execute(request)
    column_names = result.keys()
    print(" | ".join(column_names))
    for row in result:
        print(row)

measurement_concept_id | measurement_name | measurement_count
(4298391, 'Arterial blood pressure', 9750)
(4108289, 'Non-invasive mean arterial pressure', 4895)
(4354252, 'Non-invasive systolic arterial pressure', 4884)
(4068414, 'Non-invasive diastolic arterial pressure', 4879)
(36716965, 'Non-invasive blood pressure', 1262)
(4154790, 'Diastolic blood pressure', 6)
(4152194, 'Systolic blood pressure', 6)


In [116]:
request = text("""
SELECT 
    co.condition_concept_id,
    c.concept_name AS condition_name,
    COUNT(*) AS condition_count
FROM 
    demo_cdm.condition_occurrence co
JOIN 
    demo_cdm.concept c
ON 
    co.condition_concept_id = c.concept_id
WHERE 
    co.condition_concept_id != -1
GROUP BY 
    co.condition_concept_id, c.concept_name
ORDER BY 
    condition_count DESC;
""")

with engine.connect() as connection:
    result = connection.execute(request)
    column_names = result.keys()
    print(" | ".join(column_names))
    for row in result:
        print(row)

condition_concept_id | condition_name | condition_count
(320128, 'Essential hypertension', 53)
(313217, 'Atrial fibrillation', 48)
(37116430, 'Acute kidney failure stage 1', 45)
(319835, 'Congestive heart failure', 39)
(319049, 'Acute respiratory failure', 31)
(439777, 'Anemia', 25)
(37394658, 'Severe sepsis', 25)
(138384, 'Acquired hypothyroidism', 23)
(40481919, 'Coronary atherosclerosis', 18)
(46273937, 'Long-term current use of anticoagulant', 18)
(435517, 'Acidosis', 16)
(4091509, 'Esophageal reflux finding', 16)
(196236, 'Septic shock', 16)
(440383, 'Depressive disorder', 16)
(46271022, 'Chronic kidney disease', 12)
(321052, 'Peripheral vascular disease', 12)
(132797, 'Sepsis', 10)
(254061, 'Pleural effusion', 10)
(319843, 'Mitral valve disorder', 10)
(432870, 'Thrombocytopenic disorder', 10)
(434894, 'Acute posthemorrhagic anemia', 10)
(437833, 'Hypokalemia', 7)
(4059280, 'Subacute hepatic necrosis', 6)
(40479576, 'Chronic diastolic heart failure', 6)
(132736, 'Bacteremia', 6)
(

In [117]:
query = text("""
SELECT 
    m.measurement_concept_id,
    c.concept_name AS measurement_name,
    m.value_as_number,
    m.unit_source_value,
    m.measurement_date
FROM 
    demo_cdm.measurement m
JOIN 
    demo_cdm.concept c
ON 
    m.measurement_concept_id = c.concept_id
WHERE 
    m.person_id = :person_id;
""")

person_id_value = 40124 

with engine.connect() as connection:
    result = connection.execute(query, {"person_id": person_id_value})
    column_names = result.keys()
    print(" | ".join(column_names))
    for row in result:
        print(row)

measurement_concept_id | measurement_name | value_as_number | unit_source_value | measurement_date
(4354252, 'Non-invasive systolic arterial pressure', Decimal('163.0'), 'mmHg', datetime.date(2130, 2, 4))
(4068414, 'Non-invasive diastolic arterial pressure', Decimal('81.0'), 'mmHg', datetime.date(2130, 2, 4))
(4108289, 'Non-invasive mean arterial pressure', Decimal('101.0'), 'mmHg', datetime.date(2130, 2, 4))
(36716965, 'Non-invasive blood pressure', Decimal('170.0'), 'mmHg', datetime.date(2130, 2, 4))
(36716965, 'Non-invasive blood pressure', Decimal('90.0'), 'mmHg', datetime.date(2130, 2, 4))
(4354252, 'Non-invasive systolic arterial pressure', Decimal('128.0'), 'mmHg', datetime.date(2130, 2, 4))
(4068414, 'Non-invasive diastolic arterial pressure', Decimal('62.0'), 'mmHg', datetime.date(2130, 2, 4))
(4108289, 'Non-invasive mean arterial pressure', Decimal('78.0'), 'mmHg', datetime.date(2130, 2, 4))
(4354252, 'Non-invasive systolic arterial pressure', Decimal('103.0'), 'mmHg', dateti

In [118]:
concept[concept['domain_id'] == 'Measurement']

,concept_id,concept_name,concept_code,domain_id,vocabulary_id,concept_class_id,standard_concept,valid_start_date,valid_end_date,invalid_reason
0,4108289,Non-invasive mean arterial pressure,251074006,Measurement,SNOMED,Observable Entity,S,2002-01-31,2099-12-31,NaN
1,4298391,Arterial blood pressure,386534000,Measurement,SNOMED,Observable Entity,S,2003-01-31,2099-12-31,NaN
7,4154790,Diastolic blood pressure,271650006,Measurement,SNOMED,Observable Entity,S,2002-01-31,2099-12-31,NaN
8,4068414,Non-invasive diastolic arterial pressure,174255007,Measurement,SNOMED,Observable Entity,S,2002-01-31,2099-12-31,NaN
9,36716965,Non-invasive blood pressure,723237002,Measurement,SNOMED,Observable Entity,S,2017-01-31,2099-12-31,NaN
11,4354252,Non-invasive systolic arterial pressure,251070002,Measurement,SNOMED,Observable Entity,S,2002-01-31,2099-12-31,NaN
15,4152194,Systolic blood pressure,271649006,Measurement,SNOMED,Observable Entity,S,2002-01-31,2099-12-31,NaN


In [119]:
request = text("""
SELECT 
    p.person_id,
    MAX(CASE WHEN sbp.measurement_concept_id IN (4152194, 4354252) THEN sbp.value_as_number END) AS systolic_blood_pressure,
    MAX(CASE WHEN dbp.measurement_concept_id IN (4154790, 4068414) THEN dbp.value_as_number END) AS diastolic_blood_pressure
FROM 
    demo_cdm.person p
JOIN 
    demo_cdm.measurement sbp
ON 
    p.person_id = sbp.person_id
AND 
    sbp.measurement_concept_id IN (4152194, 4354252) -- Systolic Blood Pressure Concepts
AND 
    sbp.value_as_number > 140
JOIN 
    demo_cdm.measurement dbp
ON 
    p.person_id = dbp.person_id
AND 
    dbp.measurement_concept_id IN (4154790, 4068414) -- Diastolic Blood Pressure Concepts
AND 
    dbp.value_as_number > 90
GROUP BY 
    p.person_id;
""")

with engine.connect() as connection:
    result = connection.execute(request)
    column_names = result.keys()
    print(" | ".join(column_names))
    for row in result:
        print(row)

person_id | systolic_blood_pressure | diastolic_blood_pressure
(41795, Decimal('181.0'), Decimal('101.0'))
(42066, Decimal('156.0'), Decimal('142.0'))
(40687, Decimal('158.0'), Decimal('92.0'))
(42135, Decimal('153.0'), Decimal('92.0'))
(44222, Decimal('176.0'), Decimal('120.0'))
(43870, Decimal('155.0'), Decimal('109.0'))
(43879, Decimal('150.0'), Decimal('97.0'))
(43746, Decimal('180.0'), Decimal('111.0'))
(42302, Decimal('188.0'), Decimal('96.0'))
(40277, Decimal('153.0'), Decimal('116.0'))
(40595, Decimal('196.0'), Decimal('105.0'))
(43827, Decimal('189.0'), Decimal('133.0'))
(42275, Decimal('170.0'), Decimal('114.0'))
(40124, Decimal('178.0'), Decimal('105.0'))
(41976, Decimal('159.0'), Decimal('115.0'))
(42321, Decimal('184.0'), Decimal('93.0'))
(42430, Decimal('213.0'), Decimal('113.0'))
(42075, Decimal('11647.0'), Decimal('140.0'))
(44154, Decimal('205.0'), Decimal('100.0'))
(42281, Decimal('162.0'), Decimal('107.0'))
(44083, Decimal('148.0'), Decimal('95.0'))
(42367, Decimal('